In [1]:
import sys
import pandas as pd
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import homogeneity_score,completeness_score,v_measure_score,accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import RFE

TRAIN_TEST_RATIO = 0.25

In [2]:

data = pd.read_csv('dermatology.data.txt', sep=",", header = None)

# [1-11,34] podaci pre analiza
# [12-33] podaci nakon analiza
data.columns = cols = ["erythema", "scaling", "definite borders", "itching","koebner phenomenon",
                "polygonal papules","follicular papules","oral mucosal involvement","knee and elbow involvement",
                "scalp involvement","family history","melanin incontinence","eosinophils in the infiltrate","PNL infiltrate",
                "fibrosis of the papillary dermis","exocytosis","acanthosis","hyperkeratosis","parakeratosis","clubbing of the rete ridges",
               "elongation of the rete ridges","thinning of the suprapapillary epidermis","spongiform pustule","munro microabcess","focal hypergranulosis",
               "disappearance of the granular layer","vacuolisation and damage of basal layer","spongiosis","saw-tooth appearance of retes","follicular horn plug",
               "perifollicular parakeratosis","inflammatory monoluclear inflitrate","band-like infiltrate","Age","des_class"]

# data.head()

#izbaciti redove koje sadrze '?' u koloni za godine
# print(len(data))
data = data[data.Age != '?']
print(len(data))
data = data[data.des_class != 6]
print(len(data))

#izbaciti godine za probu

# data = data.drop('Age',1)


results = data['des_class']

#normalize




cols = data.columns

min_max_scaler = MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)


data.columns = cols

attributes = data.drop('des_class',1)
results.head()
# print(attributes[attributes.columns[1]])
# attributes.head()
# print(len(results))

358
338


0    2
1    1
2    3
3    1
4    3
Name: des_class, dtype: int64

In [3]:

train_data, test_data,train_results,test_results = train_test_split(attributes, results ,test_size=TRAIN_TEST_RATIO)


print('%d - %d' % (len(train_results), len(test_results)))


253 - 85


In [4]:
#izbaciti atrbiute dobijene biopsijom
# for i in range(12,33):
#     train_data = train_data.drop(cols[i],1)
#     test_data = test_data.drop(cols[i],1)

# naci najbitinije feature
model = ExtraTreesClassifier(n_estimators=250)
model.fit(data, results)
print(model.feature_importances_)
# cols = deepcopy(train_data.columns)
# print(len(cols))
# for idx,elem in enumerate(model.feature_importances_):
    
#     if float(elem) < float(0.01):
#         print(cols[idx])
       
#         train_data = train_data.drop(cols[idx],1)
#         test_data = test_data.drop(cols[idx],1)


model = LogisticRegression()
rfe = RFE(model, 10)
fit = rfe.fit(data, results)
print(fit.n_features_)
print(fit.support_)
print(fit.ranking_)

cols = deepcopy(data.columns)

print(len(cols))

# for idx,elem in enumerate(fit.support_):
    
#     if not elem:
#         print(elem) 
#         print(idx)
#         print(cols[idx])
       
#         train_data = train_data.drop(cols[idx],1)
#         test_data = test_data.drop(cols[idx],1)




[  5.53504635e-03   1.23007509e-02   9.55746336e-03   1.66815529e-02
   3.19384199e-02   3.71729575e-02   1.68359698e-03   2.19937594e-02
   2.53839277e-02   1.59659107e-02   2.26897858e-03   3.16334120e-02
   7.44390750e-03   2.56875855e-02   5.89063653e-02   2.56571661e-02
   4.17989970e-03   2.87965601e-03   9.50744763e-03   7.44934387e-02
   6.61281313e-02   5.06828158e-02   3.83445461e-03   1.53408517e-02
   3.24036483e-02   1.88260189e-02   4.73269435e-02   5.19369615e-02
   3.18135707e-02   2.87104699e-04   6.29010729e-05   2.44019786e-03
   4.20822566e-02   2.54745212e-03   2.13415448e-01]
10
[False False False False  True  True False False False False False False
 False False  True False False False False  True  True  True False False
 False False False  True  True False False False  True False  True]
[16  5 14  2  1  1 21  8 10 12 24  9 13  3  1  6 15 23 22  1  1  1 19 17 11
  7  4  1  1 25 26 18  1 20  1]
35


In [7]:
kmeans = SVC(kernel='rbf', probability=True)
neigh = KNeighborsClassifier(n_neighbors=10)
gnb = GaussianNB()

kmeans.fit(train_data,train_results)
neigh.fit(train_data,train_results)
gnb.fit(train_data,train_results)

# vclf = VotingClassifier(estimators=[('kmeans', kmeans), ('neigh', neigh), ('gnb', gnb)], voting='soft',
#                         weights=[1,1,1])
# vclf.fit(train_data,train_results)

GaussianNB()

In [8]:
# predicted = vclf.predict(test_data)
pred1 = kmeans.predict(test_data)
pred2 = neigh.predict(test_data)
pred3 = gnb.predict(test_data)
# print(accuracy_score(predicted,test_results))


import sklearn.metrics as sm
# print(sm.confusion_matrix(test_results,predicted))
print(sm.confusion_matrix(test_results,pred1))
print(sm.confusion_matrix(test_results,pred2))
print(sm.confusion_matrix(test_results,pred3))


[[32  0  0  0  0]
 [ 0 10  0  0  0]
 [ 0  0 19  0  0]
 [ 0 10  0  4  0]
 [ 0  0  0  0 10]]
[[32  0  0  0  0]
 [ 0  9  0  1  0]
 [ 0  0 19  0  0]
 [ 0  2  0 12  0]
 [ 0  0  0  0 10]]
[[32  0  0  0  0]
 [ 0  3  0  6  1]
 [ 0  0 19  0  0]
 [ 0  0  0 14  0]
 [ 0  0  0  0 10]]
